In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from google.colab import drive
from numpy.random import seed

In [2]:
drive.mount('/gdrive')
%cd /gdrive/MyDrive/F21DL_CW2/Data/CSV

Mounted at /gdrive
/gdrive/MyDrive/F21DL_CW2/Data/CSV


In [3]:
X_train = pd.read_csv('x_train_gr_smpl.csv')
X_test = pd.read_csv('x_test_gr_smpl.csv')

X = pd.concat([X_train, X_test]).reset_index().drop(columns = ['index'])

y_train = pd.read_csv('y_train_smpl.csv')
y_test = pd.read_csv('y_test_smpl.csv')

y = pd.concat([y_train, y_test]).reset_index().drop(columns = ['index'])

X_train_4000, X_test_4000, y_train_4000, y_test_4000 = train_test_split(X, y, test_size=0.55, random_state=42, stratify = y)
X_train_9000, X_test_9000, y_train_9000, y_test_9000 = train_test_split(X, y, test_size=0.95, random_state=42, stratify = y)

X_train_4000 = X_train_4000.reset_index().drop(columns = ['index'])
X_test_4000 = X_test_4000.reset_index().drop(columns = ['index'])
y_train_4000 = y_train_4000.reset_index().drop(columns = ['index'])
y_test_4000 = y_test_4000.reset_index().drop(columns = ['index'])

X_train_9000 = X_train_9000.reset_index().drop(columns = ['index'])
X_test_9000 = X_test_9000.reset_index().drop(columns = ['index'])
y_train_9000 = y_train_9000.reset_index().drop(columns = ['index'])
y_test_9000 = y_test_9000.reset_index().drop(columns = ['index'])

X_train_tensor = tf.reshape(np.array(X_train), shape = (9690, 48, 48, 1))
X_test_tensor = tf.reshape(np.array(X_test), shape = (3090, 48, 48, 1))
X_train_4000_tensor = tf.reshape(np.array(X_train_4000), shape = (5750, 48, 48, 1))
X_test_4000_tensor = tf.reshape(np.array(X_test_4000), shape = (7030, 48, 48, 1))
X_train_9000_tensor = tf.reshape(np.array(X_train_9000), shape = (639, 48, 48, 1))
X_test_9000_tensor = tf.reshape(np.array(X_test_9000), shape = (12141, 48, 48, 1))

In [4]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state = 42)
es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, min_delta = 0.0001, mode = 'min', restore_best_weights = True)

In [11]:
conv_kernels = [4, 3]
mp_kernels = [3, 2]

num_feature_maps = [64, 128, 256]

flattened_layers = [[512, 128], [1024, 512, 128], [2048, 1024, 512, 128]]

In [6]:
params_4000 = list()
acc_4000 = list()
class_reports_4000 = list()
%cd Data_from_cnn_tuning/testing_4000/random/

/gdrive/My Drive/F21DL_CW2/Data/CSV/Data_from_cnn_tuning/testing_4000/random


In [15]:
for iter in range(200):
  seed(iter)
  b1 = np.random.uniform(low = 0.85, high = 0.95)
  b2 = np.random.uniform(low = 0.9, high = 1)
  r = np.random.uniform(low = 0.001, high = 0.1)
  adm = tf.keras.optimizers.Adam(learning_rate=r, beta_1=b1, beta_2=b2)
  ck = np.random.choice(conv_kernels)
  mk = np.random.choice(mp_kernels)
  nf1 = np.random.choice(num_feature_maps)
  nf2 = np.random.choice(num_feature_maps)
  l = np.random.choice(flattened_layers)

  model = tf.keras.Sequential([tf.keras.layers.Conv2D(filters = nf1, kernel_size = (ck, ck), activation='relu', input_shape=(48, 48, 1))])
  model.add(tf.keras.layers.MaxPooling2D((mk, mk)))
  model.add(tf.keras.layers.Conv2D(nf2, (ck, ck), activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D((mk, mk)))
  model.add(tf.keras.layers.Flatten())

  for n in l:
    model.add(tf.keras.layers.Dense(n, activation='relu'))

  model.add(tf.keras.layers.Dense(10, activation='softmax'))

  model.compile(optimizer=adm,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  params_4000.append([b1, b2, r, ck, mk, nf1, nf2, l])
  print(params_4000[-1])

  model.fit(x=X_train_4000_tensor,y=y_train_4000,epochs=5, callbacks = [es], verbose = 0)

  _, accu = model.evaluate(X_test_4000_tensor, y_test_4000, verbose = 0)

  acc_4000.append(accu)

  y_pred = model.predict(X_test_4000_tensor, verbose = 0)
  y_pred_bool = np.argmax(y_pred, axis=1)
  cr = pd.DataFrame(classification_report(y_test_4000, y_pred_bool, output_dict=True)).T
  auc_p_c = list()

  for i in range(10):
    fpr, tpr, _ = roc_curve(np.array(y_test_4000), y_pred_bool, pos_label=i)
    auc_p_c.append(auc(fpr, tpr))

  cr['auc'] = auc_p_c + [0, 0, 0]
  cr.loc['weighted avg', 'auc'] = (cr.loc[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 'auc'] * cr.loc[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 'support']).sum() / cr.loc['macro avg', 'support']
  cr.loc['macro avg', 'auc'] = (cr.loc[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 'auc']).mean()
  class_reports_4000.append(cr)

  print(accu)

  cr.to_csv('testing_4000_b1_' + str(b1) + '_b2_' + str(b2) + '_lr_' + str(r) + '_ck_' + str(ck) + '_mk_' + str(mk) + '_nf1_' + str(nf1) + '_nf2_' + str(nf2) + '_l_' + str(l) + '.csv')

[0.9048813503927324, 0.971518936637242, 0.06067357423109275, 3, 2, 128, 128, [2048, 1024, 512, 128]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.23001421988010406
[0.8917022004702574, 0.9720324493442158, 0.0010113231069171437, 3, 2, 64, 64, [1024, 512, 128]]
0.9132290482521057
[0.8935994902142004, 0.9025926231827891, 0.05541658530999221, 4, 2, 64, 256, [1024, 512, 128]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.23470839858055115
[0.9050797902574576, 0.9708147822618105, 0.02979956915238149, 4, 2, 128, 256, [1024, 512, 128]]


...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.23470839858055115
[0.9109801587863418, 0.995891001231963, 0.031460812459465165, 4, 3, 128, 64, [2048, 1024, 512, 128]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.23001421988010406
[0.9154791694634146, 0.9982039284769509, 0.04059056511260089, 3, 3, 64, 256, [1024, 512, 128]]
0.23470839858055115


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
params_csv = pd.DataFrame(params_4000)
params_csv.columns = ['b1', 'b2', 'lr', 'ck', 'mk', 'nf1', 'nf2', 'l']
acc_csv = pd.DataFrame(acc_4000)
acc_csv.columns = ['acc']

In [17]:
params_csv.to_csv('params.csv')
acc_csv.to_csv('acc.csv')

In [20]:
print(max(acc_4000))
print(params_4000[acc_4000.index(max(acc_4000))])

0.9418207406997681
[0.9247905516496353, 0.9108732319900903, 0.001097492864956782, 4, 2, 128, 128, [1024, 512, 128]]


In [19]:
print(min(acc_4000))
print(params_4000[acc_4000.index(min(acc_4000))])

0.20654338598251343
[0.9467029839013676, 0.9547232249175722, 0.09729575163652356, 3, 3, 256, 128, [2048, 1024, 512, 128]]


In [ ]:
%cd ../testing_9000